In [1]:
import pandas as pd
import numpy as np
from csdi import CSDI_base
from utils import *
from training import train, evaluate
import warnings 
from preprocessing import SegmentStandardScaler
import datetime
warnings.filterwarnings('ignore')

/Users/artyomkraevskiy/opt/anaconda3/envs/DSPD/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('AirQuality.csv', sep=';')
df = df[df.columns[:-2]]

df = df[:5000]

In [3]:
df['date'] =  np.arange(0,len(df),1)

In [4]:
df = df.drop(columns = ['Date', 'Time'])

In [5]:
df= transform_format(df, segments = df.columns[:-1]).sort_values(['timestamp', 'segment']).reset_index(drop = True)
df['timestamp'] = df['timestamp'].astype(int)
df['target'] = df['target'].astype(str).str.replace(',','.',regex=True).astype(float)

share_of_misses = 0.1

df_train_nm = df[df.timestamp < 4000].reset_index(drop = True)
df_test_nm = df[df.timestamp >= 4000].reset_index(drop = True)


missing_values = np.random.choice(df.index.values, round(share_of_misses * len(df)))
df['target'] = np.where(df.index.isin(missing_values), np.nan, df['target'] )

df_train = df[df.timestamp < 4000].reset_index(drop = True)
df_test = df[df.timestamp >= 4000].reset_index(drop = True)

sc = SegmentStandardScaler(segments=df.segment.unique())
df_train = sc.fit_transform(df_train)

In [7]:
cust_df_train = Custom_Dataset(df_train)
# cust_df_test = Custom_Dataset(df_test)

In [8]:
df_train_load = DataLoader(cust_df_train, batch_size=32, shuffle=1)

In [9]:
CSDI = CSDI_base(target_dim = 13, emb_time_dim=32, diffusion=32)

In [10]:
train(CSDI,
      df_train_load,
      epochs= 20,
      learning_rate= 0.005
      )

100%|██████████| 125/125 [02:12<00:00,  1.06s/it, avg_epoch_loss=0.0991, epoch=19]


In [11]:
import torch

def custom_imputer(model, df):

    df_final = pd.DataFrame()

    base_shape = df.gt_masks.shape
    transformed_shape = (base_shape[0], base_shape[2], base_shape[1])

    df_gt_mask_T = torch.tensor(df.gt_masks.reshape(transformed_shape), dtype=torch.float32)
    df_obs_data_T = torch.tensor(df.observed_values.reshape(transformed_shape), dtype=torch.float32)
    df_obs_mask_T = torch.tensor(df.observed_masks.reshape(transformed_shape), dtype=torch.float32)
    df_times = torch.tensor(df.observed_times, dtype=torch.float32)
 
    side_info = model.get_side_info(df_times, df_gt_mask_T)

    res = model.impute(df_obs_data_T, df_gt_mask_T, df_times, side_info, 1).reshape(base_shape)

    df_final = pd.concat((df_final, pd.DataFrame(res[0])))

    for i in range(1, len(res)):
        p = res[i]
        df_final = pd.concat((df_final, pd.DataFrame(p[-1]).T))
        
    return df_final

In [12]:
cust_df_test = Custom_Dataset(df_test)

In [13]:
imputed_values = custom_imputer(CSDI, cust_df_test)

: 